In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    W1 = 0.2*init.orthogonal_(torch.empty(d1, d0, device=device), gain=1.0)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.orthogonal_(torch.empty(d2, d1, device=device), gain=1.0)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.orthogonal_(torch.empty(d3, d2, device=device), gain=1.0)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.orthogonal_(torch.empty(d4, d3, device=device), gain=1.0)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    W5 = 0.2*init.orthogonal_(torch.empty(d5, d4, device=device), gain=1.0)
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    W6 = 0.2*init.orthogonal_(torch.empty(d6, d5, device=device), gain=1.0)
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "Orthogonal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.5286130905151367 - sq_loss: 661.6843872070312 - tot_loss: 922.2364956646816 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 2 / 500 
 - time: 1.4654662609100342 - sq_loss: 294.0819396972656 - tot_loss: 482.30420965049416 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 3 / 500 
 - time: 1.4816415309906006 - sq_loss: 164.40667724609375 - tot_loss: 262.13626398239285 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 4 / 500 
 - time: 1.4714395999908447 - sq_loss: 89.64942169189453 - tot_loss: 146.10560800880194 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.5000660419464111 - sq_loss: 48.30636215209961 - tot_loss: 83.86332913918886 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1

Repeatition 1 Epoch 48 / 500 
 - time: 1.477961778640747 - sq_loss: 0.0004037957696709782 - tot_loss: 1.1406581981791533 - acc: 0.5496463547334058 - val_acc: 0.5307091957923312
Repeatition 1 Epoch 49 / 500 
 - time: 1.4511003494262695 - sq_loss: 0.00037843812606297433 - tot_loss: 1.098040443370337 - acc: 0.5531828073993471 - val_acc: 0.5317271801832372
Repeatition 1 Epoch 50 / 500 
 - time: 1.443232774734497 - sq_loss: 0.00035502167884260416 - tot_loss: 1.0696935787736948 - acc: 0.5602557127312296 - val_acc: 0.5334238208347472
Repeatition 1 Epoch 51 / 500 
 - time: 1.4591917991638184 - sq_loss: 0.000333524018060416 - tot_loss: 1.0321283058065092 - acc: 0.5710010881392819 - val_acc: 0.5395317271801833
Repeatition 1 Epoch 52 / 500 
 - time: 1.465620756149292 - sq_loss: 0.00031350846984423697 - tot_loss: 0.9979093781967094 - acc: 0.5888193688792165 - val_acc: 0.5493722429589413
Repeatition 1 Epoch 53 / 500 
 - time: 1.455549716949463 - sq_loss: 0.0002950486377812922 - tot_loss: 0.97293771

Repeatition 1 Epoch 95 / 500 
 - time: 1.4356205463409424 - sq_loss: 3.0232244171202183e-05 - tot_loss: 0.28654047358133994 - acc: 0.876088139281828 - val_acc: 0.8673227010519172
Repeatition 1 Epoch 96 / 500 
 - time: 1.4465663433074951 - sq_loss: 2.868156116164755e-05 - tot_loss: 0.28569222350961354 - acc: 0.8774483133841132 - val_acc: 0.8683406854428232
Repeatition 1 Epoch 97 / 500 
 - time: 1.432927131652832 - sq_loss: 2.7204438083572313e-05 - tot_loss: 0.280415955688909 - acc: 0.8782644178454843 - val_acc: 0.8693586698337292
Repeatition 1 Epoch 98 / 500 
 - time: 1.4243364334106445 - sq_loss: 2.5811974410316907e-05 - tot_loss: 0.2697822927032121 - acc: 0.8792165397170838 - val_acc: 0.8741092636579573
Repeatition 1 Epoch 99 / 500 
 - time: 1.4573888778686523 - sq_loss: 2.4491311705787666e-05 - tot_loss: 0.2640446807145054 - acc: 0.8797606093579978 - val_acc: 0.8754665761791652
Repeatition 1 Epoch 100 / 500 
 - time: 1.4093577861785889 - sq_loss: 2.3258626242750324e-05 - tot_loss: 0.

Repeatition 1 Epoch 141 / 500 
 - time: 3.029378890991211 - sq_loss: 3.5430805382929975e-06 - tot_loss: 0.1467586909085412 - acc: 0.9129488574537541 - val_acc: 0.9104173736002714
Repeatition 1 Epoch 142 / 500 
 - time: 3.008884906768799 - sq_loss: 3.420994744374184e-06 - tot_loss: 0.15331415332251197 - acc: 0.9139009793253536 - val_acc: 0.9100780454699695
Repeatition 1 Epoch 143 / 500 
 - time: 3.017299175262451 - sq_loss: 3.3134845125459833e-06 - tot_loss: 0.1385817355958352 - acc: 0.9147170837867247 - val_acc: 0.9104173736002714
Repeatition 1 Epoch 144 / 500 
 - time: 3.0847885608673096 - sq_loss: 3.2006889796321047e-06 - tot_loss: 0.1522245694003459 - acc: 0.9151251360174102 - val_acc: 0.9110960298608755
Repeatition 1 Epoch 145 / 500 
 - time: 2.996100664138794 - sq_loss: 3.09304573420377e-06 - tot_loss: 0.15530893896542608 - acc: 0.9152611534276387 - val_acc: 0.9114353579911775
Repeatition 1 Epoch 146 / 500 
 - time: 2.9594171047210693 - sq_loss: 2.991633891724632e-06 - tot_loss: 0

Repeatition 1 Epoch 187 / 500 
 - time: 3.0255179405212402 - sq_loss: 1.3407886854110984e-06 - tot_loss: 0.11346356542452618 - acc: 0.941512513601741 - val_acc: 0.9351883271123176
Repeatition 1 Epoch 188 / 500 
 - time: 3.111921548843384 - sq_loss: 1.3279453696668497e-06 - tot_loss: 0.10311707004420612 - acc: 0.9426006528835691 - val_acc: 0.9358669833729216
Repeatition 1 Epoch 189 / 500 
 - time: 3.1706066131591797 - sq_loss: 1.3158942238078453e-06 - tot_loss: 0.10925614103391812 - acc: 0.9431447225244831 - val_acc: 0.9368849677638276
Repeatition 1 Epoch 190 / 500 
 - time: 3.0684165954589844 - sq_loss: 1.3033854884270113e-06 - tot_loss: 0.11712574793010333 - acc: 0.9432807399347116 - val_acc: 0.9372242958941296
Repeatition 1 Epoch 191 / 500 
 - time: 3.0556585788726807 - sq_loss: 1.2916873401991324e-06 - tot_loss: 0.11474114994690598 - acc: 0.9440968443960827 - val_acc: 0.9372242958941296
Repeatition 1 Epoch 192 / 500 
 - time: 3.0410678386688232 - sq_loss: 1.2827632644984988e-06 - to

...

Repeatition 1 Epoch 279 / 500 
 - time: 3.0946178436279297 - sq_loss: 8.748576760808646e-07 - tot_loss: 0.09718687451675767 - acc: 0.9717083786724701 - val_acc: 0.9531727180183237
Repeatition 1 Epoch 280 / 500 
 - time: 3.069347620010376 - sq_loss: 8.736712970858207e-07 - tot_loss: 0.10086669649781999 - acc: 0.9718443960826986 - val_acc: 0.9528333898880217
Repeatition 1 Epoch 281 / 500 
 - time: 2.9804999828338623 - sq_loss: 8.708981908966962e-07 - tot_loss: 0.08650402528795875 - acc: 0.9723884657236126 - val_acc: 0.9531727180183237
Repeatition 1 Epoch 282 / 500 
 - time: 3.1008715629577637 - sq_loss: 8.69393772973126e-07 - tot_loss: 0.09675531680911642 - acc: 0.9721164309031556 - val_acc: 0.9528333898880217
Repeatition 1 Epoch 283 / 500 
 - time: 3.089242458343506 - sq_loss: 8.687250101502286e-07 - tot_loss: 0.10582303118390213 - acc: 0.9723884657236126 - val_acc: 0.9528333898880217
Repeatition 1 Epoch 284 / 500 
 - time: 2.9937307834625244 - sq_loss: 8.65926210735779e-07 - tot_loss: 

Repeatition 1 Epoch 325 / 500 
 - time: 3.0187320709228516 - sq_loss: 7.805278983141761e-07 - tot_loss: 0.09399055722500793 - acc: 0.9793253536452666 - val_acc: 0.9521547336274178
Repeatition 1 Epoch 326 / 500 
 - time: 2.981391429901123 - sq_loss: 7.79130459704902e-07 - tot_loss: 0.09394713738740479 - acc: 0.9793253536452666 - val_acc: 0.9528333898880217
Repeatition 1 Epoch 327 / 500 
 - time: 3.0765180587768555 - sq_loss: 7.771963055347442e-07 - tot_loss: 0.0839058379986577 - acc: 0.9794613710554951 - val_acc: 0.9524940617577197
Repeatition 1 Epoch 328 / 500 
 - time: 3.0391862392425537 - sq_loss: 7.745898642497195e-07 - tot_loss: 0.09219471149305547 - acc: 0.9795973884657236 - val_acc: 0.9538513742789277
Repeatition 1 Epoch 329 / 500 
 - time: 3.026585817337036 - sq_loss: 7.729501589892607e-07 - tot_loss: 0.09522512110330883 - acc: 0.9797334058759521 - val_acc: 0.9538513742789277
Repeatition 1 Epoch 330 / 500 
 - time: 3.080198049545288 - sq_loss: 7.708844691478589e-07 - tot_loss: 0

Repeatition 1 Epoch 371 / 500 
 - time: 3.077850103378296 - sq_loss: 7.14149621217075e-07 - tot_loss: 0.08656328819278714 - acc: 0.9823177366702938 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 372 / 500 
 - time: 2.994126081466675 - sq_loss: 7.126209879970702e-07 - tot_loss: 0.09263888093543793 - acc: 0.9823177366702938 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 373 / 500 
 - time: 3.12716007232666 - sq_loss: 7.107223609637003e-07 - tot_loss: 0.0958466877071138 - acc: 0.9823177366702938 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 374 / 500 
 - time: 3.095583915710449 - sq_loss: 7.090894769135048e-07 - tot_loss: 0.09352330656194541 - acc: 0.9824537540805223 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 375 / 500 
 - time: 3.0900466442108154 - sq_loss: 7.073867323015293e-07 - tot_loss: 0.09442487869392258 - acc: 0.9824537540805223 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 376 / 500 
 - time: 3.1329264640808105 - sq_loss: 7.058570190565661e-07 - tot_loss: 0.0

Repeatition 1 Epoch 417 / 500 
 - time: 2.9461450576782227 - sq_loss: 6.551978231073008e-07 - tot_loss: 0.09957708519156816 - acc: 0.9836779107725789 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 418 / 500 
 - time: 3.0539186000823975 - sq_loss: 6.544232746819034e-07 - tot_loss: 0.09449368177096562 - acc: 0.9838139281828074 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 419 / 500 
 - time: 3.0305769443511963 - sq_loss: 6.534215231113194e-07 - tot_loss: 0.0901773841821617 - acc: 0.9838139281828074 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 420 / 500 
 - time: 3.0203170776367188 - sq_loss: 6.52615938179224e-07 - tot_loss: 0.08665400547942859 - acc: 0.983949945593036 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 421 / 500 
 - time: 2.9622740745544434 - sq_loss: 6.518433792734868e-07 - tot_loss: 0.1018832309620139 - acc: 0.983949945593036 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 422 / 500 
 - time: 3.0637423992156982 - sq_loss: 6.493563660114887e-07 - tot_loss: 0

Repeatition 1 Epoch 463 / 500 
 - time: 2.9866292476654053 - sq_loss: 6.085755330786924e-07 - tot_loss: 0.09352543483394693 - acc: 0.984766050054407 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 464 / 500 
 - time: 3.058588981628418 - sq_loss: 6.07278650477383e-07 - tot_loss: 0.11136916922445661 - acc: 0.984766050054407 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 465 / 500 
 - time: 3.1504311561584473 - sq_loss: 6.057525752112269e-07 - tot_loss: 0.09146111830224235 - acc: 0.984766050054407 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 466 / 500 
 - time: 3.1356041431427 - sq_loss: 6.051200784895627e-07 - tot_loss: 0.09467607740453288 - acc: 0.984766050054407 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 467 / 500 
 - time: 3.052912712097168 - sq_loss: 6.048713316886278e-07 - tot_loss: 0.09890168631193053 - acc: 0.984766050054407 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 468 / 500 
 - time: 3.0914626121520996 - sq_loss: 6.040992275302415e-07 - tot_loss: 0.09702

In [8]:
seed

10